<a href="https://colab.research.google.com/github/nafissa1-web/nafissa/blob/main/Weather_station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1: import libraries

In [67]:
import requests #this lib is 4 fetching data from api
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #convert catogerical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #for classification & regression
from sklearn.metrics import mean_squared_error #measures the error
from datetime import datetime, timedelta #to handle date & time
import pytz #lib for time zone

In [68]:
API_KEY = '195f1783d83d96a46b04ffc3be2031e5'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'


**1. Fetch current weather data**

> Add blockquote



In [69]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #f"" easily insert variables into a string
    #this statement constuct the API request url
    response = requests.get(url) #send a GET request to the API endpoint
    data = response.json() #parse the response as JSON
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'humidity': round(data['main']['humidity']),
        'description': data['main'][0]['description'],
        'country': data['IT']['country'],
        'WindGustDir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'WindGustSpeed': data['wind']['speed'],

    }
#

# *2.Read historical data*

> Add



In [70]:
def read_historical_data(filename):
  df = pd.read_csv(filename) #upload csv file into data frame
  df = pd.dropna() #removes missing values raws
  df = pd.drop_duplicates() #removes duplicate rows
  return df


# 3.prepare data for training




In [71]:
def prepare_data(data):
  le = LabelEncoder() #categories into nmbrs windgus& rain tomorrow are categorical in our code
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])
  #define feature variable & target variables
  X = data[['MinTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] # Corrected: Removed duplicate 'MinTemp'
  y = data['RainTomorrow'] #target which were trying to predict
  return X, y, le

# ***4.Train Rain Prediction model***





In [72]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #20% for testing. randm state ensures we split same nbr each time
  model = RandomForestClassifier(n_estimators=100, random_state= 42) #n_estimators=100 means we used 100 decision tree
  model.fit(X_train, y_train)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print('Mean Squared Error is :')
  print(mean_squared_error(y_test, y_pred))
  return model #to use it later

#Prepare regression data

In [73]:
#regression helps us finding predictions for continuos data

In [74]:
def prepare_regression_data(data, feature):
  x, y = [], [] #list x takes the feature values & y 4 the target values
  #we create a data set where having pairs from x & its equivalent value of y
  for i in range(len(data) - 1): #4 iterations for 5 raws, x0 takes raw 1's value. y4 takes last raw value.
      x.append(data[feature].iloc[i])
      y.append(data[feature].iloc[i+1])
  x = np.array(x).reshape(-1, 1) #convert a list to 2D array with 1 column (remember -1 means dont care abt nbr)
  #cuz the graph of regression is on 2D plane
  y = np.array(y) #the target
  return x, y

#**Train regression data**


In [75]:
def train_regression_model(x, y):   #data that we fetched it from the dataset & future which specifies the column that we are
  modal = RandomForestRegressor(n_estimators=100, random_state=42) #100 trees
  modal.fit(x, y)
  return modal


#Predict feature

In [76]:
def predict_future(modal, current_value):
  predictions = [current_value]
  for i in range(5):
    # Reshape the scalar current_value to a 2D array (1 sample, 1 feature)
    next_value = modal.predict(np.array(predictions[-1]).reshape(1, -1))
    predictions.append(next_value[0])
  return predictions[1:] # Return predictions excluding the initial current_value

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Weather Analysis Function

In [78]:
API_KEY = '195f1783d83d96a46b04ffc3be2031e5'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code != 200:
        print(f"Error fetching data for {city}: {data.get('message', 'Unknown error')}")
        return None

    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'], # Corrected: Access description from 'weather' list
        'country': data['sys']['country'], # Corrected: Access country from 'sys'
        'WindGustDir': data['wind']['deg'],
        'pressure': data['main']['pressure'] if 'pressure' in data['main'] else None, # Added check for pressure
        'WindGustSpeed': data['wind']['speed'] if 'speed' in data['wind'] else None, # Added check for speed
    }

def read_historical_data(filename):
  df = pd.read_csv(filename) #upload csv file into data frame
  df = df.dropna() #removes missing values raws
  df = df.drop_duplicates() #removes duplicate rows
  return df

def wather_view(): #using this we will pass data into our functions
  city = input('Enter city name: ')
  current_weather = get_current_weather(city)

  if current_weather is None:
      return # Exit if API call failed

  # !!! IMPORTANT: Update this path to the correct location of your weather.csv file
  # For example, if it's in your Google Drive under 'Colab Notebooks' folder:
  # historical_data = read_historical_data('/content/drive/MyDrive/Colab Notebooks/weather.csv')
  historical_data= read_historical_data('/content/weather.csv') # Placeholder, please update this line
  x, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(x, y)

  #convert wind direction from nbrs to categories
  wind_deg = current_weather['WindGustDir'] % 360 if current_weather['WindGustDir'] is not None else 0 # Handle None for WindGustDir
  compass_points = [
      ("N", 0, 11.25), ('NNE', 11.25, 33.75), ('NE',33.75, 56.25),
      ('ENE', 56.25, 78.75), ('E', 78.75, 101.25), ('ESE', 101.25, 123.75),
       ('SE', 123.75, 146.25), ('SSE', 146.25, 168.75), ('S', 168.75, 191.25),
      ('SSW', 191.25, 213.75), ('SW', 213.75, 236.25), ('WSW', 236.25, 258.75),
      ('W', 258.75, 281.25), ('WNW', 281.25, 303.75), ('NW', 303.75, 326.25),
      ('NNW', 326.25,348.75)
  ]
  #now we check the direction of wind
  compass_direction = next((point for point, start, end in compass_points if start<= wind_deg <end), "N") # Default to N if not found
  # Check if le.classes_ contains compass_direction before transforming
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1 # If not in classes, assign -1 or handle as appropriate

  # Define the feature list in the correct order, matching X from prepare_data
  feature_cols_for_prediction = ['MinTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']

  current_data_dict = {
      'MinTemp' : current_weather['temp_min'],
      'WindGustDir' : compass_direction_encoded,
      'WindGustSpeed': current_weather['WindGustSpeed'] if current_weather['WindGustSpeed'] is not None else 0, # Handle None
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'] if current_weather['pressure'] is not None else 1000, # Handle None, provide a default
      'Temp': current_weather['current_temp']
  }

  # Create DataFrame ensuring column order matches training data
  current_df = pd.DataFrame([current_data_dict], columns=feature_cols_for_prediction)


  #rain prediction
  rain_prediction = rain_model.predict(current_df)[0]

  #preapare a regression modal for temp & humidity
  temp_x, temp_y = prepare_regression_data(historical_data, 'Temp')
  temp_model = train_regression_model(temp_x, temp_y)
  hum_x, hum_y = prepare_regression_data(historical_data, 'Humidity')
  hum_model = train_regression_model(hum_x, hum_y)

  #predict future tem & hum
  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_hum = predict_future(hum_model, current_weather['humidity'])

  #prepare time for futur predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime('%H:00') for i in range(5)] #next 5 times

  #display results

  print(f"City: {city}, {current_weather['country']}")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['current_temp']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print('\nFuture Temperature Predictions')

  for time, temp in zip(future_times, future_temp):
     print(f'{time}: {round(temp,1)}°C')

  print('\nFuture Humidity Predictions')
  for time, humidity in zip(future_times, future_hum):
     print(f'{time}: {round(humidity,1)}%')
wather_view()

Enter city name: England
Mean Squared Error is :
0.1780821917808219
City: England, US
Feels Like: 20°C
Current Temperature: 20°C
Minimum Temperature: 19°C
Maximum Temperature: 20°C
Humidity: 94%
Weather Prediction: scattered clouds
Rain Prediction: Yes

Future Temperature Predictions
21:00: 25.1°C
22:00: 20.2°C
23:00: 16.2°C
00:00: 15.4°C
01:00: 15.6°C

Future Humidity Predictions
21:00: 62.2%
22:00: 56.5%
23:00: 40.7%
00:00: 52.9%
01:00: 51.0%
